### 发展史
循环神经网络(Recurrent Neural Network, RNN)一般是指时间递归神经网络，主要用于对序列数据进行建模。

教程博文地址：https://leovan.me/cn/2018/09/rnn/

总之呢，循环神经网络就是输入序列数据，前面隐藏层的输出会输入到下一层。

好玩的应用

利用深度学习创作音乐

https://magenta.tensorflow.org/

画画
https://magenta.tensorflow.org/assets/sketch_rnn_demo/index.html

### RNN
循环神经网络的基本结构特别简单，就是将网络的输出保存在一个记忆单元中，这个记忆单元和下一次的输入一起进入神经网络中。我们可以看到网络在输入的时候会联合记忆单元一起作为输入，网络不仅输出结果，还会将结果保存到记忆单元中。

RNN 可以被看做是同一神经网络的多次赋值，每个神经网络模块会把消息传递给下一个。

网络中具有循环结构，这也是循环神经网络名字的由来，同时根据循环神经网络的结构也可以看出它在处理序列类型的数据上具有天然的优势。因为网络本身就是 一个序列结构，这也是所有循环神经网络最本质的结构。

循环神经网络具有特别好的记忆特性，能够将记忆内容应用到当前情景下，但是网络的记忆能力并没有想象的那么有效。记忆最大的问题在于它有遗忘性，我们总是更加清楚地记得最近发生的事情而遗忘很久之前发生的事情，循环神经网络同样有这样的问题。

pytorch 中使用 nn.RNN 类来搭建基于序列的循环神经网络，它的构造函数有以下几个参数：

- input_size：输入数据X的特征值的数目。 
- hidden_size：隐藏层的神经元数量，也就是隐藏层的特征数量。
- num_layers：循环神经网络的层数，默认值是 1。 
- bias：默认为 True，如果为 false 则表示神经元不使用 bias 偏移参数。
- batch_first：如果设置为 True，则输入数据的维度中第一个维度就是 batch 值，默认为 False。默认情况下第一个维度是序列的长度， 第二个维度才是 - - batch，第三个维度是特征数目。
- dropout：如果不为空，则表示最后跟一个 dropout 层抛弃部分数据，抛弃数据的比例由该参数指定。

RNN 中最主要的参数是 input_size 和 hidden_size，这两个参数务必要搞清楚。其余的参数通常不用设置，采用默认值就可以了。

In [11]:
import torch

In [5]:
rnn = torch.nn.RNN(20, 50, 2)
input = torch.randn(100, 32, 20)
h_0 = torch.randn(2, 32, 50)
output,hn=rnn(input ,h_0) 
print(output.size(),hn.size())

torch.Size([100, 32, 50]) torch.Size([2, 32, 50])


手动实现RNN

In [6]:
class RNN(object):
    def __init__(self,input_size,hidden_size):
        super().__init__()
        self.W_xh=torch.nn.Linear(input_size,hidden_size) #因为最后的操作是相加 所以hidden要和output 的shape一致
        self.W_hh=torch.nn.Linear(hidden_size,hidden_size)
        
    def __call__(self,x,hidden):
        return self.step(x,hidden)
    def step(self, x, hidden):
        #前向传播的一步
        h1=self.W_hh(hidden)
        w1=self.W_xh(x)
        out = torch.tanh( h1+w1)
        hidden=self.W_hh.weight
        return out,hidden

In [7]:
rnn = RNN(20,50)
input = torch.randn( 32 , 20)
h_0 =torch.randn(32 , 50) 
seq_len = input.shape[0]

In [10]:
input.shape

torch.Size([32, 20])

In [8]:
for i in range(seq_len):
    output,hn= rnn(input[i, :], h_0)
print(output.size(),h_0.size())

torch.Size([32, 50]) torch.Size([32, 50])


### LSTM
是Long Short Term Memory Networks的缩写，即长的短时记忆系统。
![](https://miro.medium.com/max/1400/1*goJVQs-p9kgLODFNyhl9zA.gif)
标准的循环神经网络内部只有一个简单的层结构，而LSTM内部有4个层结构：

第一层是忘记层：决定状态中丢弃什么信息

第二层tanh层用来产生更新值的候选项，说明状态在某些维度上需要加强，在某些维度上需要减弱

第三层sigmoid层，它的输出值要乘到tanh层的输出上，起到一个缩放的作业，极端情况下sigmoid输出0说明相应维度上的状态不需要更新

最后一层决定输出什么，输出值跟状态有关。候选项中哪些部分最终会被输出由最后一个sigmoid层决定

pytorch中使用nn.LSTM类来搭建基于序列的循环神经网络，参数与RNN类似

In [13]:
lstm = torch.nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, hn = lstm(input, (h0, c0))
print(output.size(),hn[0].size(),hn[1].size())
output.shape

torch.Size([5, 3, 20]) torch.Size([2, 3, 20]) torch.Size([2, 3, 20])


torch.Size([5, 3, 20])

In [14]:
?torch.nn.LSTM